<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Basics 03 - Board Calibration

This **notebook** is a part of the the **SLab** documentation.

It implements all the procedures to calibrate the SLab board.  
Code is provided for full calibration or for calibration restricted to 4 ADCs.

Version 1.5 (2/4/2019)  
License information is at the end of the document

## Introduction

At this point you are supposed to have a working **Hardware Board**. And you should know that it is working. Now we will try to guarantee that we can get good measurements from the system. 

In a measurement system the most important feature is to obtain repetitive measurements. That is, every time we measure the same real magnitude, we should get the same reading. Having a good read value is not as important as having always the same value.

Let's say we have a measurement system that for real voltage values between 0V and 3V gives voltage readings between 0.2V and 2.2V like in the figure below:

![Calibration Curve](images\Basics_03\Calibration_Curve.png)

For instance, if the real voltage is 1V, the read value will be 1,35V. That's a big error but it is not a measurement problem. As long as we always get the same input to output relationship, we can always use the curve to deduce the real value from any measured value. That's why it is called a **calibration curve**. Using this curve we can calibrate our readings to reconstruct the real values.

In practice, we never get exactly the same values when we measure the same magnitudes. The following figure shows a more real case where there is some uncertainty to the measurement values. We have superposed different colors for different measurements using the instruments. The uncertainty from one measurement to the next could be due to noise or drifts in the instrument.

![Uncertainty](images\Basics_03\Uncertainty.png)

The uncertainty is what limits the precision we can get from the instrument as the systematic measurement to real value relationship can always be calibrated.
In general, it is best to have a nonlinear low uncertainty instrument than a linear high uncertainty instrument as uncertainty cannot be calibrated. That way, the following will be a bad instrument because, although it is linear, the response has a lot of random content.

![Bad Instrument](images\Basics_03\Bad_Instrument.png)

In a laboratory setup we usually deal with systematic errors with calibration and we deal with uncertainty errors with the averaging of several measurements.

In this document we will obtain the calibration curves for the **ADCs** and **DACs** so that we correct the systematic errors in the measurements of our circuits.

## Ratiometric measurements

In the SLab system we use DACs to set voltages and ADCs to measure voltages. Both ADCs and DACs are ratiometric devices. That means that the input to a DAC and the output of an ADC is given as a unitless ratio to a reference voltage. Although it is not always true, most microcontroller boards use its Vdd voltage as the reference. The SLab system holds the both the value of the supply Vdd voltage and the Vref reference voltage for the ADCs and DACs although they are usually similar. 

We can define the ADC and DAC ratios as:

$\qquad ratio = \frac{Voltage}{Vref}$

Where **Vref** is the reference voltage for the **ADCs** and **DACs** that is usually similar or equal to the supply voltage **Vdd**.

In a n bit DAC, the ratio is set as a DACin number from $0$ to $2^{n-1}$ respect to $2^n$. So, the voltage $V_{DAC}$ generated by a DAC is:

$\qquad V_{DAC}=V_{Ref} \cdot DAC_{ratio} = 
V_{Ref} \frac{DAC_{in}}{2^n} \qquad 0 \leq DAC_{in} < 2^n$

The F303RE board, for instance, includes two 12 bit DACs. Input on those DACs can be from 0 to 4095, so the maximum output voltage value will be:

$\qquad V_{DACmax} = V_{ref}\frac{2^{12}-1}{2^{12}}=3.3V\frac{4095}{4096}=3.299V$

In a similar way, in a n bit ADC, the measured voltage VADC is provided as a ratio against the Vref reference. 

$\qquad ratio_{max} = \frac{2^{12}-1}{2^{12}} = 0,9998$

In practice it is so close to one that the SLab commands set this maximum value when we request an impossible ratio of 1.0.

## Slab Calibration Explained

Calibration in the SLab system uses three elements:

* Vref voltage (Reference for ADCs and DACs)
* Ratiometric DAC curves   CalDACi(x)
* Ratiometric ADC curves   CalADCi(x)

For each ADC number i, from 1 to 4, it shall hold:

$\qquad \frac{Measured Voltage}{Vref}=CalADC_i \left( \frac{RealVoltage}{V_{Ref}} \right)$

So the calibration curve of the ADCs indicates the ratiometric value we measure on the ADC when we set each possible real ratiometric value at the input of the ADC.

In a similar way, for each DAC number **i**, it shall hold:

$\qquad \frac{Real Voltage}{Vref}=CalDAC_i \left( \frac{SetVoltage}{V_{Ref}} \right)$

So the calibration curves for the DACs indicates the ratiometric real value set for each intended ratio we set the DAC to generate.

All operations on the DACs and ADCs use the three calibration elements so that we always work with the real values. Slab uses Vref and CalADCi(x) to obtain real values for measured ADC values and uses Vref and CalDACi(x) to set the DAC inputs from the intended real value to set at the DAC outputs.

In order for the calibration system to operate correctly we need to guarantee two conditions:

* That calibration is **always the same**. That is, each time the buffer circuit or the hardware board changes we need to recalibrate. It is also advisable to calibrate from time to time to correct from temporal deviations. Ideally you should calibrate at the start of each measurement session but that is not always practical.


* That calibration curves are **monotonous**. That is, that you can invert the curves to obtain real voltages from measured voltages in ADCs and set voltages from real voltages in DACs.

As DACs and ADCs are quite linear, the SLab system currently uses calibration curves with only 9 points. Data is linear interpolated between points.

Hardware boards are not guaranteed to contain any reliable voltage reference, so we will need a measurement instrument capable of reading voltages to perform the calibration. A hand held multimeter usually will do. From this point we will call this instrument a **Voltage Metter** (VM).

One way to calibrate a system is to perform a **chain of calibrations**. First, we calibrate one instrument, and then we use the measurements from that instrument to calibrate other instruments. Each calibrated instrument can be used to calibrate any uncalibrated instrument.

Calibration in our case is performed in **three stages**. 

In the **first stage** we calibrate the first DAC, **DAC1** and all ADCs against an external **VM**. We also obtain the **Vref** and **Vdd** values in this stage.

In the **second stage** we calibrate the other DACs (only DAC2 on the **F303 board** for instance) against the previously calibrated ADCs. 

In the **third stage** we check the calibration. This stage is optional, because the calibration is complete at the end of the second stage. But verifying the calibration won't hurt.

By default, calibration is performed on all available channels. Some **SLab** boards feature a large number of ADC channels and, perhaps, you don't need to calibrate all the channels. In general, for most **SLab** experiments, you only need to use a maximum of four **ADC** channels. We will provide code in this notebook for two options: 

* Full calibration of all ADCs
* Calibration of only 4 ADCs

Select the option that suits your needs. Of course, you can change the number of calibrated channels to any other number as long as the number of ADCs is bigger or equal that the number of available DACs.

In previous **SLab** versions, there was a four stage calibration procedure. This method is still available, but only for backward compatibility. This document only describes the new **three stage** calibration method.

## Connecting and Setting the Calibration Path

This **notebook** runs on folder inside the **Jupyter** folder that is inside the main **SLab** root folder. We would like to use always the same **configuration files**. In order to do that, we must instruct **SLab** to set an specific path **prefix** that, in this case, points to the [Files Folder](http://localhost:8888/tree/Files).

This **general prefix** will affect all **data** files associated to **SLab** including the **calibration** files.

We can set also a **calibration prefix**, specific for the calibration data, that will be added to the previous **general preffix**. By default the **calibration prefix** will point to the [Calibrations Folder](http://localhost:8888/tree/Files/Calibrations) inside the [Files Folder](http://localhost:8888/tree/Files).

**Run** the following code to import the **SLab** module, set the **general** SLab file path **prefix**, set the **calibration prefix** and connect to the board. 

In [ ]:
import slab                                     # Import the SLab module

boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

Before continuing, **wait** until the connection completes and the output says that you are connected to a board.  
If you have already performed the calibration, it will show on the text. in that case, you can jump to the **Calibration Check** section to check the current calibration. If it is ok, you don't need to repeat the calibration.

Note that **SLab** cannot check if the calibration data loaded was obtained from the **Hardware Board** you are using as the **Board Firmware** don't report, yet, a serial number. Using calibration data from other board can give worse results that having no calibration data a all. So, in case of doubt, repeat the calibration.

## First calibration stage

We will first calibrate the **Vdd** voltage, and all **ADC** curves. 

Any instrument needs to be calibrated against another better calibrated instrument. We will use a **Voltage Metter** (VM) instrument capable of measuring voltages to perform this calibration stage. A **handheld multimeter** will do. If you cannot get your hands on any voltage measuring instrument you can skip the calibration. You should note, however, that skipping the calibration will degrade the precision of any measurement you perform in the SLab system.

You could perform only the second calibration stage, than don't need the multimeter. The calibration will be better, for the DACs, that no calibration at all. But far worse than a full calibration.

Once the system is calibrated, you don't need the multimeter any more until another calibration is performed.

To perform the **Vdd** and **ADCs** calibration, you will need no connect all the ADC inputs together (8 in the SLab Zero 303 setup). Those inputs shall be connected to the **positive** terminal of the **VM** and the **negative** terminal of the **VM** shall be connected to the board **ground** node.

Then we will make, in sequence, **three** different connections to the ADC and VM node and other three nodes. 

The first case is just connecting the board **vdd(3V3)** to this node. Rememeber that will rarely use the **5V** power. So, when we talk about **Vdd**, it means **3.3V**, al least on current SLab systems. The following figure shows the circuit diagram for a 8 channel calibration. 

![cal1a](images\Basics_03\cal1a.png)


Observe that the above diagram has 10 square boxes, so you will connect 10 **harware board** male pins to the **breadboard** using 10 **Female to Male** Jump Wires. If you follow the **SLab style**, each Jump Wire will have the color of the box it connects to.

After the first measurement of the **Vdd** voltage you will be instructed to disconnect **Vdd** and connect the node that joins the **ADCs** and the **VM** to ground. You will be asked to provide the **VM** reading although it will most probably be zero. Again, we show a diagram for an 8 channel calibration.

![cal1b](images\Basics_03\cal1b.png)

Finally you will be instructed to disconnect **GND** from the node and connect it to the **DAC1** output as the following figure shows. Again, a 8 channel calibration diagram is shown.

![cal1a](images\Basics_03\cal1c.png)

In this last circuit, several measurements will be performed.

Run the following **code** and follow its instructions. Remember to select the box adequate for the number of channels you want to calibrate.

You should see a two **calibration graphs** when the calibration ends. One for **DAC1** and another for all **ADC** curves. 

---

The following code cell calibrates **all available** ADC channels:

In [ ]:
# Run this code for a full all ADC channels calibration

# First calibration stage 
slab.newCalibrate1()

---

The following code cell calibrates only the **first four** ADC channels:

In [ ]:
# Run this code for a full a 4 ADC channels calibration

# First calibration stage 
slab.newCalibrate1(na=4)

---

We should see good calibration curves. It is possible that **DAC1** is not capable to reach 0 and 1, but this is normal. 

See that four files have been generated in the [Calibrations Folder](http://localhost:8888/tree/Files/Calibrations):

* **Cal_Vdd.dat** contains the reference **Vref** and the supply **Vdd** values
* **Cal_DAC.dat** contains the **DAC 1** preliminar ratiometric calibration curve
* **Cal_ADC.dat** contains all the **ADC** ratiometric calibration curves
* **Cal_Log.txt** contains the log of this and any previous calibrations

Next time you connect to the board those files will be loaded so the calibration is persistent in the system.

## Second calibration stage

The first calibration stage calibrates all **ADCs**. As ADCs, specially the ones that use opamp drivers, are usually not subject to loading effect, their values will be more reliables than the ones on the DACs.

The **DAC 1** calibration guarantees that when we set DAC 1 to output 1V it really generates 1V output and not 0,8V or 1,2V. In practice it won't be exactly 1V but it will be close enough for our experiments. 

Using the **ADCs** as a reference, we can now calibrate the rest of the **DACs**.

The script will instruct you to connect all **DACs** from number 2 to the ADCs with the same number. As the **F303 Board** only features two DACs, that means implementing the following circuit.

![cal2](images\Basics_03\cal2.png)

This stage is independent on the number of ADC channels you are calibrating.

To perform the second calibration stage, make the connections and run the **next code**. 

In [ ]:
# Second calibration stage
slab.newCalibrate2()

If you use the **F303RE Board**, unless it has been modified as explained in [this document](http://localhost:8888/notebooks/Boards/Nucleo_F303RE_Zero/Board%2003%20-%20DAC2%20Improvement.ipynb), the **DAC2** calibration curves should be worse than the one on **DAC1**. This is due to the loading effect of the **user LED** that is connected to the **DAC2** output.

Observe also that a file **"Cal_ADC.dat"** has been updated in the [Calibrations Folder](http://localhost:8888/tree/Files/Calibrations). This file now holds all the DAC calibration curves.

## See Calibration Curves

You can check again the calibration curves whenever you want. In order to be able to zoom on the curves, we can use the following **code cell** that instruct **SLab** and **Jupyter** to use interactive plots.

In [ ]:
# Make plots interactive
slab.interactivePlots()
%matplotlib notebook

Now, running the following code cell you can see both the **ratiometric** and the **voltage** curves that relate the **Real Voltages** on the **ADC inputs** with the **Values measured** by the ADCs.  
Note that any non calibrated **ADC** channel will show the default $measure = real$ calibration curve.

Every time you read a voltage using an **ADC**, **SLab** will use the curve to obtain the **Real Volatge** that corresponds to the **Measured Voltage** for this particular **ADC**.

In [ ]:
# Show ADC calibration Data
slab.showADCcal()

The following code cell shows the equivalent of the above curves for the **DACs**. You can see both the **ratiometric** and the **voltage** curves that relate the **Programmed Voltages** on the **ADC inputs** with the **Real Voltages** generated on the DAC outputs.  

Every time you set a voltage using a **DAC**, **SLab** will use the curve to obtain the voltage it should **program** so that the **DAC** generates the expected **Real Voltage** for this particular **DAC**. Off course, if the **DAC** cannot generate a certain voltage because it is out of the **Real** range of values it can generate, having a calibration curve does not help.

In [ ]:
slab.showDACcal()

## Calibration Check

In order to end the calibration procedure it is an optional but wise idea to check the calibration.

To perform the check, connect each DAC output to the ADC input with the same number. Then, connect the remaining ADC channels to DAC 1 output. Again, the figure shown is for a 8 ADC channels calibration.

![cal3](images\Basics_03\cal3.png)

Then, run the following **code**. It will take a while because a lot of measurements are performed with a high number of reads to provide average values.

We will enter interactive mode so that you can **zoom** in the curves if you want to. Again, select the code box that suits the number of ADC channels you are calibrating.

---

Execute the following code cell to check the calibration with all **ADC** channels.

In [ ]:
# Run this box for a full all ADC channels calibratuion check

# Make plots interactive
slab.interactivePlots()
%matplotlib notebook

# Check the calibration
slab.checkCalibration(pause=False)

---

Execute the following code cell to check the calibration with only the first **four ADC** channels.

In [ ]:
# Run this box for a 4 ADC channels calibratuion check

# Make plots interactive
slab.interactivePlots()
%matplotlib notebook

# Check the calibration
slab.checkCalibration(pause=False,na=4)

If the calibration is successful you should get a set of curves that feature, more or less, the same value in the horizontal and vertical axes. For each measurement point **SLab** is using the **calibration curves** to compensate for the systematic errors of the **ADCs** and **DACs**.

If executed from a console, after drawing the curve and before you dismiss it, the script also sets both DACs to 1.0V. If you want, you can use a multimeter to check their voltage so that you are confident that not only the ratiometric calibration curves are good but also that we use a proper value of the reference voltage. In Jupyter this won't work, just use the following code to set the **DAC** voltages.

You can modify the voltages if you please to check other values.

The SLab system doesn't pretend to be high precision so a less than 10 mV error can be considered enough.

In [ ]:
# Set DAC voltages
slab.setVoltage(1,1)
slab.setVoltage(2,1)

# Read ADC voltages
print('ADC1 voltage is',slab.readVoltage(1),'V')
print('ADC2 voltage is',slab.readVoltage(2),'V')

After measuring those voltages using the multimeter, you can set the **DACs** to a safe zero voltage using the code below.

In [ ]:
# Set DACs to zero
slab.zero()

## DAC calibration limits

Once the system is calibrated, you can expect the ADC inputs to give the proper readings if the voltages you are measuring are inside the calibrated region between the board GND and Vdd voltages (or between Vss and Vdd if the hardware board can work with negative voltages).

The same not always hold true for the DAC output channels. The voltages generated by the DACs are usually buffered but the buffer circuits have always a limit in their current capabilities if you use **small resistance** loads.

If you expect to find this kind of **loading** problems, it is recommended to connect an ADC line on the node driven by the DAC to measure the **real voltage** on the node as it can be different to the set value if the load has low enough resistance.

## Managing several boards

Probably you only have one **SLab** board. If this is the case, you can skip this section.

If you have several **SLab** boards, the calibration is specific for each one of them. That means that you need to use different calibration files for each board.

The management is easy if you create a new folder inside the **SLab\Code\Calibrations** folder for each one of the boards and you put the three calibration files inside this folder.

One of my **SLab** boards is the **Fat Board \#1**. Let's call it **"Fat1"**. I can create a **Fat1** folder inside the **Calibrations** folder and put the three calibration files inside.

Then, in order to connect with the board I can use the **following code**. That way, the proper calibration files are loaded.

In [ ]:
import slab                                     # Import slab if it has not already been done
boardFolder = 'Fat1/'                           # We calibrate for the 'Fat1' board
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

## Recalibration of the SLab system

Calibration is not permanent. It starts to drift as soon as you complete it due to component aging and effects of ambient variables like the temperature. Ideally you should calibrate the board at the start of each work session. In practice you don't need to calibrate as often as our experiments don't demand high precision.

At any time you can check the calibration by running the **checkCalibration()** function or by executing the code cell in the **Check Calibration** section of this document. If you observe that any curve is not good you can recalibrate the system.

All SLab experiments assume that you are using a properly calibrated hardware board and that you use buffers for the ADCs and DACs. If your board is not calibrated, qualitative measurements will be ok, errors will be bigger than in a properly calibrated board. 

That's why the **SLab Standard Docking** for the DAC and ADC wires is like in the image below.

![Docking](images\Basics_03\docking.jpg)

Observe that **DAC1** is connected to **ADC1**, **ADC3** and **ADC4** and also **DAC2** is connected to **ADC2**. That way you don't need to move the wires from the docking position to check the calibration.

Note that, if your boad features more than two DACs or more than four ADCs, those lines will not be checked using the standard docking. This is usually no problem as most SLab projects are designed so that they don't need more than two DACs or four ADCs.

In order to check the board you can check the calibration using the following call that restricts the curves to only four ADC channels.

In [ ]:
# Check the calibration for four ADC channels
slab.checkCalibration(pause=False,na=4)

## Calibration History

There is no good measurement instrument if there is no good calibration. When you use a measurement system it is good to know when and who calibrated it.

In the [PDF folder](http://localhost:8888/tree/PDF) you can find a [SLab Calibration Sheet](http://localhost:8888/files/PDF/SLab%20Calibration%20Sheet.pdf) that you can print. You can use one sheet for each board you own so you can keep track of their calibrations. The calibration history could be important because most times you will not calibrate all ADC channels and it is a good idea to know if the las board calibration included all the channels you want to use.

In fact, the [SLab Calibration Sheet](http://localhost:8888/files/PDF/SLab%20Calibration%20Sheet.pdf) is an **Acrobat Form** document. If you go to the folder:

$\qquad$ `[SLab]\Jupyter\PDF`

Where `[SLab]` is the **Root SLab folder** and open the **SLab Calibration Sheet** with **Acrobat** you can fill the boxes on the sheet so that the calibration informatuion is kept in the **PDF** file. Thay way you don't really need to print the file and the calibration data will always be on the pendrive.

Off course you are not required to print or use this calibration sheet, but taking note of the calibration history of a measurement instrument is always good practice.

If you are lazy and don't fill-up the calibration sheet, you can always check the calibration log for the board. All the calibration data is in the [Calibrations Folder](http://localhost:8888/tree/Files/Calibrations). If you use only one board, the calibration log file [Cal_Log.txt](http://localhost:8888/edit/Files/Calibrations/Cal_Log.txt) shows the calibration history for the board. If you use more than one board, there should be one **Cal_Log.txt** file for each boad folder in the [Calibrations Folder](http://localhost:8888/tree/Files/Calibrations). 

This file is interesting because it is automatically generated during the calibration and shows, on the last lines, how many ADC channels were calibrated on the last calibration.

That's all for now. This also ends the setup of the system so now you have a complete calibrated measurement environment and you can start working with real circuits. Next chapter will deal with the **SLab workflow** that we will use on our experiments.

You can use the following code to **disconnect** now from the board.

In [ ]:
# Disconnect from the board
slab.disconnect()

## References

**SLab Python References**   
Those are the reference documents for the SLab Python modules. They describe the commands that can be carried out after importing each module. 
They should be available in the [References](http://localhost:8888/tree/Reference) folder.

**TinyCad**  
Circuit images on this document have been drawn using the free software TinyCad  
https://sourceforge.net/projects/tinycad/

**SciPy**  
All the functions plots have been generated using the Matplotlib SciPy package.  
https://www.scipy.org/


## Document license

Copyright  ©  Vicente Jiménez (2018-2019)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">